In [1]:
# install packages from requirements.txt
!pip install -r requirements.txt

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to c:\users\tpeiq\appdata\local\temp\pip-req-build-8n5_jxpw
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit fb2becacbf3c48249ce960f3ef60a862f8fc6707
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to c:\users\tpeiq\appdata\local\temp\pip-install-71jxck97\elegantrl_6328e23710c349c7b4acb6f7b224c71e
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit b2ec84b19a82e6f18b781743d8bf37ca27134238
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git 'C:\Users\tpeiq\AppData\Local\Temp\pip-req-build-8n5_jxpw'
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git 'C:\Users\tpeiq\AppData\Local\Temp\pip-install-71jxck97\elegantrl_6328e23710c349c7b4acb6f7b224c71e'


In [1]:
# SMART LIVE TRADING BOT SCRIPT (DAILY)
# Supports multi-stock, multi-agent SMART model, and Tiger Brokers API with balance checks, logging, and capital-aware execution

import pandas as pd
import numpy as np
import yfinance as yf
import logging
import itertools
import os
from datetime import datetime, timedelta
from stable_baselines3 import PPO, A2C, SAC

# === Setup Logging ===
date_str = datetime.now().strftime('%Y-%m-%d')
log_path = f'./log/demo_smart_trading_bot_{date_str}.log'
error_log_path = f'./log/demo_smart_trading_bot_error_{date_str}.log'
logging.basicConfig(
    filename=log_path,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
# Improved logging: log INFO to 'smart_trading_bot.log', ERROR to 'smart_trading_bot_error.log'
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# File handler for INFO and above
info_handler = logging.FileHandler(log_path)
info_handler.setLevel(logging.INFO)
info_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
info_handler.setFormatter(info_formatter)

# File handler for ERROR and above
error_handler = logging.FileHandler(error_log_path)
error_handler.setLevel(logging.ERROR)
error_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
error_handler.setFormatter(error_formatter)

# Remove default handlers and add our handlers
if logger.hasHandlers():
    logger.handlers.clear()
logger.addHandler(info_handler)
logger.addHandler(error_handler)

logging.info("SMART Trading Bot started.")



In [2]:
import os
import tigeropen
print(os.listdir(os.path.dirname(tigeropen.__file__)))

['common', 'examples', 'fundamental', 'push', 'quote', 'tiger_open_client.py', 'tiger_open_config.py', 'trade', '__init__.py', '__pycache__']


In [3]:
# === Tiger Brokers API ===
from tigeropen.tiger_open_config import TigerOpenClientConfig
from tigeropen.common.consts import Language, OrderType
from tigeropen.trade.trade_client import TradeClient

# Fallback enum for TimeInForce if missing
class TimeInForce:
    DAY = 'DAY'
    GTC = 'GTC'
    IOC = 'IOC'

class AccountType:
    INDIVIDUAL = 'INDIVIDUAL'
    JOINT = 'JOINT'
    CORPORATE = 'CORPORATE'

# === Your trained DRL models ===
most_common_model_df = pd.read_csv('most_common_model.csv', sep=',', header=0)
print(most_common_model_df.head())
last_most_common = most_common_model_df.iloc[-1]['most_common'].upper()
print("Last row's most_common:", last_most_common)

if last_most_common == 'PPO':
    model_best = PPO.load("trained_models/agent_best_model")
elif last_most_common == 'A2C':
    model_best = A2C.load("trained_models/agent_best_model")
elif last_most_common == 'SAC':
    model_best = SAC.load("trained_models/agent_best_model")



# === Stocks to monitor ===
tickers = ['aapl', 'amd', 'amzn', 'cat', 'crwd', 'googl', 'gs', 'hd', 'ibm',
       'intc', 'meta', 'msft', 'nvda', 'pypl', 't', 'tsla', 'v']

# === Parameters ===
hmax = 20  # Max shares per trade per asset
TRADE_END_DATE = datetime.today().strftime("%Y-%m-%d")
TRAIN_START_DATE = (datetime.today() - timedelta(days=365*3)).strftime("%Y-%m-%d")
daily_loss_threshold = 0.05
capital_limit_pct = 0.8
print(TRADE_END_DATE)
print(TRAIN_START_DATE)


  most_common  confidence
0         a2c         NaN
1         a2c         NaN
2         a2c    0.494033
3         a2c    0.444181
4         sac    0.437055
Last row's most_common: A2C
2025-08-11
2022-08-12


c:\Users\tpeiq\anaconda3\envs\tradingbot310\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [4]:
"""
# === Preprocess with YahooDownloader and FeatureEngineer ===
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.config import INDICATORS

df_raw = YahooDownloader(
    start_date=TRAIN_START_DATE,
    end_date=TRADE_END_DATE,
    ticker_list=tickers
).fetch_data()


# Step 2: Get the last available date
last_date = df_raw['date'].max()

# Convert last_date to pandas Timestamp if it's a string
if isinstance(last_date, str):
	last_date_dt = pd.to_datetime(last_date)
else:
	last_date_dt = last_date

# Step 3: Filter all rows from the last date (usually one per ticker)
last_day_data = df_raw[df_raw['date'] == last_date].copy()

# Step 4: Duplicate and increment the date by 1 day
next_day = last_date_dt + pd.Timedelta(days=1)
last_day_data['date'] = next_day.strftime('%Y-%m-%d') if isinstance(last_date, str) else next_day

# Step 5: Append the duplicated rows to the original data
df_patched = pd.concat([df_raw, last_day_data], ignore_index=True)

fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=INDICATORS,
    use_vix=True,
    use_turbulence=True,
    user_defined_feature=False
)
processed = fe.preprocess_data(df_patched)

# Align to all dates and tickers
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(), processed['date'].max()).astype(str))
combination = list(itertools.product(list_date, list_ticker))
processed_full = pd.DataFrame(combination, columns=["date", "tic"]) \
    .merge(processed, on=["date", "tic"], how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])
processed_full = processed_full.fillna(0)

# Make sure 'date' column is in datetime format
processed_full['date'] = pd.to_datetime(processed_full['date'])

# Sort first for consistency
processed_full = processed_full.sort_values(by=['date', 'tic']).reset_index(drop=True)

# Assign the same index to all rows with the same date
processed_full.index = processed_full.groupby('date').ngroup()

"""

'\n# === Preprocess with YahooDownloader and FeatureEngineer ===\nfrom finrl.meta.preprocessor.yahoodownloader import YahooDownloader\nfrom finrl.meta.preprocessor.preprocessors import FeatureEngineer\nfrom finrl.config import INDICATORS\n\ndf_raw = YahooDownloader(\n    start_date=TRAIN_START_DATE,\n    end_date=TRADE_END_DATE,\n    ticker_list=tickers\n).fetch_data()\n\n\n# Step 2: Get the last available date\nlast_date = df_raw[\'date\'].max()\n\n# Convert last_date to pandas Timestamp if it\'s a string\nif isinstance(last_date, str):\n\tlast_date_dt = pd.to_datetime(last_date)\nelse:\n\tlast_date_dt = last_date\n\n# Step 3: Filter all rows from the last date (usually one per ticker)\nlast_day_data = df_raw[df_raw[\'date\'] == last_date].copy()\n\n# Step 4: Duplicate and increment the date by 1 day\nnext_day = last_date_dt + pd.Timedelta(days=1)\nlast_day_data[\'date\'] = next_day.strftime(\'%Y-%m-%d\') if isinstance(last_date, str) else next_day\n\n# Step 5: Append the duplicated r

In [5]:
# print(processed_full.date.max())

In [6]:
"""
# === Build environment ===
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv 

stock_dimension = len(tickers)
state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

env = StockTradingEnv(df=processed_full, **env_kwargs)
obs, _ = env.reset()
initial_account_value = env.initial_amount

"""

'\n# === Build environment ===\nfrom finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv \n\nstock_dimension = len(tickers)\nstate_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension\nprint(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")\n\nbuy_cost_list = sell_cost_list = [0.001] * stock_dimension\nnum_stock_shares = [0] * stock_dimension\n\nenv_kwargs = {\n    "hmax": 100,\n    "initial_amount": 1000000,\n    "num_stock_shares": num_stock_shares,\n    "buy_cost_pct": buy_cost_list,\n    "sell_cost_pct": sell_cost_list,\n    "state_space": state_space,\n    "stock_dim": stock_dimension,\n    "tech_indicator_list": INDICATORS,\n    "action_space": stock_dimension,\n    "reward_scaling": 1e-4\n}\n\nenv = StockTradingEnv(df=processed_full, **env_kwargs)\nobs, _ = env.reset()\ninitial_account_value = env.initial_amount\n\n'

In [7]:
# print("Model observation space:", model_best.observation_space.shape)

In [8]:
final_model = model_best


In [9]:
# Display the attributes and methods of TigerOpenClientConfig
print(dir(TigerOpenClientConfig))

['_TigerOpenClientConfig__get_device_id', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_get_domain_by_type', '_get_props_path', '_load_props', 'account', 'charset', 'get_token_path', 'is_paper', 'is_us', 'language', 'license', 'load_token', 'private_key', 'query_domains', 'quote_server_url', 'refresh_server_info', 'sdk_version', 'secret_key', 'server_url', 'should_token_refresh', 'sign_type', 'socket_host_port', 'store_token', 'tiger_id', 'tiger_public_key', 'timeout', 'timezone', 'token', 'token_refresh_duration']


In [10]:
# === Connect to Tiger Brokers ===
from tigeropen.common.util.signature_utils import read_private_key
from tigeropen.quote.quote_client import QuoteClient



YESTERDAY_FILE = r'./data/demo_account_value_yesterday.txt'
def get_config(sandbox=False):
    cfg = TigerOpenClientConfig(sandbox_debug=sandbox)
    cfg.private_key = read_private_key(os.getenv("TIGER_PRIVATE_KEY_PEM"))  # Path to your private key file
    cfg.tiger_id = os.getenv("TIGER_ID")
    cfg.account = os.getenv("TIGER_PAPER_ACCOUNT")  # paper account
    cfg.language = Language.en_US
    return cfg

config = get_config(sandbox=False)
quote_client = QuoteClient(config)
trade_client = TradeClient(config)

# Check quote access (optional)
quote_client.grab_quote_permission()

trade_client = TradeClient(config)
account_summary = trade_client.get_assets()
print(account_summary)


# === Real Stop-Loss Check Using Live Account ===
daily_loss_threshold = 0.05

try:
    # Get current net liquidation value from Tiger account
    assets = trade_client.get_assets()
    current_value = float(assets[0].summary.net_liquidation)
    print(f"✅ Current account value: {current_value}")

    # Read yesterday’s value (default to 0.0 if file missing or empty)
    previous_value = 0.0
    if os.path.exists(YESTERDAY_FILE):
        with open(YESTERDAY_FILE, 'r') as f:
            content = f.read().strip()
            if content:
                previous_value = float(content)
            else:
                print("⚠️ Yesterday's file is empty. Assuming previous value = 0.0")
    else:
        print("📄 Yesterday's file not found. Assuming previous value = 0.0")

    # Calculate loss ratio (only if previous_value is positive)
    if previous_value > 0:
        daily_loss = (previous_value - current_value) / previous_value
        if daily_loss > daily_loss_threshold:
            logging.warning(f"[STOP LOSS TRIGGERED] Drop: {daily_loss:.2%}, "
                            f"Prev: {previous_value}, Now: {current_value}")
            print("❌ STOP: Live account drop exceeded threshold. No trades executed.")
            exit()

    # Save current value for next run
    with open(YESTERDAY_FILE, 'w') as f:
        f.write(str(current_value))
    print(f"✅ Live account loss check passed. Current value: {current_value}, Previous value: {previous_value}")

except Exception as e:
    logging.error(f"[STOP LOSS ERROR] Failed to check live account loss: {e}")
    print(f"Error checking live account loss: {e}")



[PortfolioAccount({'account': '21127702597532756', 'summary': Account({'accrued_cash': inf, 'accrued_dividend': inf, 'available_funds': inf, 'buying_power': 4000000.0, 'cash': 1000000.0, 'currency': 'USD', 'cushion': inf, 'day_trades_remaining': inf, 'equity_with_loan': inf, 'excess_liquidity': inf, 'gross_position_value': inf, 'initial_margin_requirement': inf, 'maintenance_margin_requirement': inf, 'net_liquidation': 1000000.0, 'realized_pnl': 0.0, 'regt_equity': inf, 'regt_margin': inf, 'sma': inf, 'timestamp': None, 'unrealized_pnl': 0.0}), 'segments': defaultdict(<class 'tigeropen.trade.domain.account.Account'>, {'S': SecuritySegment({'accrued_cash': 0.0, 'accrued_dividend': 0.0, 'available_funds': 1000000.0, 'cash': 1000000.0, 'equity_with_loan': 1000000.0, 'excess_liquidity': 1000000.0, 'gross_position_value': 0.0, 'initial_margin_requirement': 0.0, 'leverage': 0.0, 'maintenance_margin_requirement': 0.0, 'net_liquidation': 1000000.0, 'regt_equity': inf, 'regt_margin': inf, 'sma'

In [11]:
from finrl.config import INDICATORS
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader


df_raw = YahooDownloader(
    start_date=TRAIN_START_DATE,
    end_date=TRADE_END_DATE,
    ticker_list=tickers
).fetch_data()

# Step 2: Get the last available date
last_date = df_raw['date'].max()

# Convert last_date to pandas Timestamp if it's a string
if isinstance(last_date, str):
	last_date_dt = pd.to_datetime(last_date)
else:
	last_date_dt = last_date

# Step 3: Filter all rows from the last date (usually one per ticker)
last_day_data = df_raw[df_raw['date'] == last_date].copy()

# Step 4: Duplicate and increment the date by 1 day
next_day = last_date_dt + pd.Timedelta(days=1)
last_day_data['date'] = next_day.strftime('%Y-%m-%d') if isinstance(last_date, str) else next_day

# Step 5: Append the duplicated rows to the original data
df_patched = pd.concat([df_raw, last_day_data], ignore_index=True)

fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=INDICATORS,
    use_vix=True,
    use_turbulence=True,
    user_defined_feature=False
)
processed = fe.preprocess_data(df_patched)

# Align to all dates and tickers
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(), processed['date'].max()).astype(str))
combination = list(itertools.product(list_date, list_ticker))
processed_full = pd.DataFrame(combination, columns=["date", "tic"]) \
    .merge(processed, on=["date", "tic"], how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])
processed_full = processed_full.fillna(0)

# Make sure 'date' column is in datetime format
processed_full['date'] = pd.to_datetime(processed_full['date'])

# Sort first for consistency
processed_full = processed_full.sort_values(by=['date', 'tic']).reset_index(drop=True)

# Assign the same index to all rows with the same date
processed_full.index = processed_full.groupby('date').ngroup()


stock_dimension = len(tickers)
state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

buy_cost_list = sell_cost_list = [0.001] * stock_dimension
# Get live positions from Tiger Broker
positions = trade_client.get_positions()

# Make sure tickers are lowercase for matching
tickers_lower = [t.lower() for t in tickers]

# Build list of shares held per ticker in the correct order
num_stock_shares = [
    next((pos.quantity for pos in positions if hasattr(pos, "contract") and pos.contract.symbol.lower() == t), 0)
    for t in tickers_lower
]

print("Live num_stock_shares:", num_stock_shares)
assets = trade_client.get_assets()
account_summary = assets[0].segments.get('S')
cash_balance = float(account_summary.available_funds)
print(f"Live cash balance: {cash_balance}")

env_kwargs = {
    "hmax": 20,
    "initial_amount": cash_balance,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

env = StockTradingEnv(df=processed_full, **env_kwargs)
obs, _ = env.reset()
initial_account_value = env.initial_amount
print(f"Initial account value: {initial_account_value}, Cash balance: {cash_balance}")

final_action = final_model.predict(obs, deterministic=True)[0]


YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (12750, 8)


[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (750, 8)
Successfully added vix


Successfully added turbulence index
Stock Dimension: 17, State Space: 171
Live num_stock_shares: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Live cash balance: 1000000.0
Initial account value: 1000000.0, Cash balance: 1000000.0


In [12]:
for ticker, shares in zip(tickers, num_stock_shares):
    print(f"{ticker}: {shares}")


aapl: 0
amd: 0
amzn: 0
cat: 0
crwd: 0
googl: 0
gs: 0
hd: 0
ibm: 0
intc: 0
meta: 0
msft: 0
nvda: 0
pypl: 0
t: 0
tsla: 0
v: 0


In [13]:
import pandas as pd

# Combine final_action and tickers into a DataFrame
action_df = pd.DataFrame({
    'ticker': tickers,
    'action': final_action
})
# Set random seed as today's date for reproducibility
np.random.seed(int(datetime.now().strftime('%Y%m%d')))
# Add a random column for secondary sorting
action_df['random'] = np.random.random(len(action_df))

# Sort by absolute value of action (descending), then by random values (ascending)
action_df['abs_action'] = action_df['action'].abs()
action_df_sorted = action_df.sort_values(['abs_action', 'random'], ascending=[False, True]).reset_index(drop=True)
action_df_sorted = action_df_sorted.drop(['random', 'abs_action'], axis=1)


# Split back to tickers and final_action arrays
tickers = action_df_sorted['ticker'].tolist()
final_action = action_df_sorted['action'].values


In [14]:
action_df

,ticker,action,random,abs_action
0,aapl,1.000000,0.952839,1.000000
1,amd,1.000000,0.199097,1.000000
2,amzn,1.000000,0.615653,1.000000
3,cat,1.000000,0.450361,1.000000
4,crwd,1.000000,0.997401,1.000000
5,googl,1.000000,0.379850,1.000000
6,gs,-1.000000,0.326740,1.000000
7,hd,1.000000,0.388965,1.000000
8,ibm,1.000000,0.710627,1.000000
9,intc,0.336953,0.228168,0.336953


In [15]:
action_df_sorted

,ticker,action
0,amd,1.000000
1,gs,-1.000000
2,googl,1.000000
3,hd,1.000000
4,cat,1.000000
5,t,1.000000
6,amzn,1.000000
7,ibm,1.000000
8,meta,-1.000000
9,tsla,-1.000000


In [16]:
print("Final action vector:", final_action)
print("Tickers in action vector:", tickers)

Final action vector: [ 1.         -1.          1.          1.          1.          1.
  1.          1.         -1.         -1.         -1.          1.
  1.          0.66133326 -0.4567479   0.3369528  -0.18670385]
Tickers in action vector: ['amd', 'gs', 'googl', 'hd', 'cat', 't', 'amzn', 'ibm', 'meta', 'tsla', 'pypl', 'aapl', 'crwd', 'msft', 'v', 'intc', 'nvda']


In [17]:
print(df_raw.date.max())

2025-08-08


In [18]:
from tigeropen.common.util.order_utils import (limit_order)           # Attached Order



# === Get account balance and positions ===
capital_limit = cash_balance * capital_limit_pct
logging.info(f"Cash balance: {cash_balance}, Capital allocation limit: {capital_limit:.2f}")
positions = trade_client.get_positions()
current_holdings = {pos.contract.symbol.lower(): pos.quantity for pos in positions if hasattr(pos, "contract") and pos.contract.symbol.lower() in tickers}
logging.info(f"Current holdings: {current_holdings}")

# === Execute Action Vector ===
def execute_trades(action_vector, tickers, trade_client, cash_balance):
    try:
        prices = {tic: yf.Ticker(tic).history(period='1d')['Close'].iloc[-1] for tic in tickers}
    except Exception as e:
        logging.error(f"Error fetching prices: {e}")
        return

    for i, a in enumerate(action_vector):
        # Refresh cash balance before each trade
        assets = trade_client.get_assets()
        account_summary = assets[0].segments.get('S')
        cash_balance = float(account_summary.available_funds)
        capital_limit = cash_balance * capital_limit_pct
        symbol = tickers[i].upper()
        logging.info(f"Processing action for {symbol}: {a:.4f}")
        print(f"Processing action for {symbol}: {a:.4f}")
        price = prices.get(symbol.lower(), None)
        contract = trade_client.get_contracts(symbol=symbol)[0]
        if price is None:
            logging.warning(f"Price not available for {symbol}, skipping.")
            continue
        logging.info(f"Current price for {symbol}: ${price:.2f}")
        print(f"Current price for {symbol}: ${price:.2f}")
        shares = int(abs(a) * hmax)
        if shares == 0:
            continue

        action_type = None
        if a > 0.01:
            cost = shares * price
            if cost > capital_limit:
                # Adjust to maximum affordable shares
                shares = int(capital_limit / price)
                cost = shares * price
                print(f"💡 Adjusted BUY for {symbol}: {shares} shares within limit (${cost:.2f})")
                if shares == 0:
                    continue
            action_type = 'BUY'

        elif a < -0.01:
            held = current_holdings.get(symbol.lower(), 0)
            if shares > held:
                shares = held  # Sell only what you have
                print(f"💡 Adjusted SELL for {symbol}: only selling {shares} shares (held: {held})")
                if shares == 0:
                    continue
            action_type = 'SELL'

        else:
            continue

        logging.info(f"{action_type} {shares} shares of {symbol} at ${price:.2f}")
        print(f"{action_type} {shares} shares of {symbol} at ${price:.2f}")
        try:
            if action_type == 'BUY':
                price = round(price * 0.995, 2)
            elif action_type == 'SELL':
                price = round(price * 1.01, 2)
            stock_order = limit_order(
                account=config.account,
                contract=contract,
                action=action_type,
                limit_price=price,
                quantity=shares
            )
            trade_client.place_order(stock_order)
            print(stock_order)
        except Exception as e:
            logging.error(f"Order failed for {symbol}: {e}")
            print(f"Order failed for {symbol}: {e}")

# === Trade ===
execute_trades(final_action, tickers, trade_client, cash_balance)

# === Save today’s account value for next day ===
try:
    updated_assets = trade_client.get_assets()
    latest_value = float(updated_assets[0].summary.net_liquidation)
    with open(YESTERDAY_FILE, 'w') as f:
        f.write(str(latest_value))
    logging.info(f"Saved latest account value for stop-loss: {latest_value}")
    print(f"Saved latest account value for stop-loss: {latest_value}")
except Exception as e:
    logging.error(f"Failed to save account value for stop-loss check: {e}")
    print(f"Failed to save account value for stop-loss check: {e}")

logging.info("SMART trade execution complete.")
print("SMART trade execution complete.")

Processing action for AMD: 1.0000
Current price for AMD: $172.76
BUY 20 shares of AMD at $172.76
Order({'account': '21127702597532756', 'id': 40108593570119680, 'order_id': None, 'parent_id': None, 'order_time': None, 'reason': None, 'trade_time': None, 'action': 'BUY', 'quantity': 20, 'filled': 0, 'avg_fill_price': 0, 'commission': None, 'realized_pnl': None, 'trail_stop_price': None, 'limit_price': np.float64(171.9), 'aux_price': None, 'trailing_percent': None, 'percent_offset': None, 'order_type': 'LMT', 'time_in_force': None, 'outside_rth': None, 'order_legs': None, 'algo_params': None, 'algo_strategy': None, 'secret_key': None, 'liquidation': None, 'discount': None, 'attr_desc': None, 'source': None, 'adjust_limit': None, 'sub_ids': [], 'user_mark': None, 'update_time': None, 'expire_time': None, 'can_modify': None, 'external_id': None, 'combo_type': None, 'combo_type_desc': None, 'is_open': None, 'contract_legs': None, 'filled_scale': None, 'total_cash_amount': None, 'filled_cash

In [19]:
from datetime import datetime
import os

# === Prepare log content ===
log_lines = []
today_str = datetime.now().strftime('%Y-%m-%d')
log_file = f"transaction_actions/demo_transaction_{today_str}.txt"

try:
    # yahoo finance data latest date
    log_lines.append(f"Yahoo finance data latest date: {df_raw.date.max()}\n")
    log_lines.append(f" \n ----------------------------------------------------------------------")
    # Current Holdings
    log_lines.append(f"Date: {today_str}\n")
    log_lines.append(f" \n ----------------------------------------------------------------------")
    log_lines.append("Current Holdings:")
    positions = trade_client.get_positions()
    for pos in positions:
        if pos.quantity > 0 and hasattr(pos, "contract") and hasattr(pos.contract, "symbol"):
            log_lines.append(f" - {pos.contract.symbol}: {pos.quantity} shares")
    logging.info("Current Holdings Completed")
    log_lines.append(f" \n ----------------------------------------------------------------------")
    # Pending Orders
    log_lines.append("\nPending Orders:")
    open_orders = trade_client.get_open_orders()
    for order in open_orders:
        if hasattr(order, 'contract') and hasattr(order.contract, 'symbol') and hasattr(order, 'action') and hasattr(order, 'order_id'):
            price_info = getattr(order, 'limit_price', 'N/A')
            status = getattr(order, 'status', 'Unknown')
            log_lines.append(f" - {order.contract.symbol}: {order.action} order (Limit: {price_info}, Qty: {order.quantity}) [Status: {status}, ID: {order.order_id}]")
    if not open_orders:
        log_lines.append(" - None")
    logging.info("Pending Orders Completed")
    log_lines.append(f" \n ----------------------------------------------------------------------")
    # Executed Orders (today only)
    log_lines.append("\nExecuted Orders (Today):")
    all_orders = trade_client.get_orders()
    for order in all_orders:
        filled_time = getattr(order, 'filled_time', None)
        if filled_time and filled_time.strftime('%Y-%m-%d') == today_str:
            log_lines.append(f"- {order.contract.symbol}: {order.action} order filled (Price: {getattr(order, 'filled_avg_price', 'N/A')}, Qty: {order.quantity}, ID: {order.order_id})")
    if not all_orders:
        log_lines.append(" - None")
except Exception as e:
    log_lines.append(f"\nError retrieving open orders or positions: {e}")
    logging.error(f"Error retrieving open orders or positions: {e}")

# === Save to file ===
try:
    if not log_lines:
        log_lines.append("⚠️ No trading data available for logging.")

    log_file_path = os.path.abspath(log_file)
    logging.info(f"Writing transaction log to: {log_file_path}")
    logging.info(f"Number of log lines: {len(log_lines)}")

    with open(log_file, 'w') as f:
        f.write('\n'.join(log_lines))

    logging.info(f"Transaction log saved: {log_file_path}")
except Exception as e:
    logging.error(f"Failed to write transaction log: {e}")